In [181]:

import pandas as pd
import langchain

In [182]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [183]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [184]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [185]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [186]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

response = get_completion(prompt)

response

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



"I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

In [187]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat



ChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, temperature=0.0, openai_api_key='sk-proj-A2oLnKF1BBXs4zdUjAQGT3BlbkFJOJvHoC2Ni9augcHa9fdm', openai_proxy='')

In [188]:
from langchain.prompts import ChatPromptTemplate
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```

"""
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n\n')

In [189]:

template_string_2 = """Write a rap song in with the pen of: {pen}"""
prompt_template_2 = ChatPromptTemplate.from_template(template_string_2)
prompt_template_2 .messages[0].prompt.input_variables
customer_messages = prompt_template_2.format_messages(
                    pen='Eminem')
chat(customer_messages).content

"(Verse 1)\nYo, it's Slim Shady back in the game\nSpitting fire, setting the world aflame\nI'm the rap god, the one and only\nMy rhymes are sick, my flow is holy\n\nI'm a lyrical genius, a mastermind\nEvery word I spit is one of a kind\nI'm the king of the game, the top of the throne\nI'll never back down, I'll never be dethroned\n\n(Chorus)\nI'm Eminem, the rap sensation\nI'll leave you in awe, in admiration\nMy words cut deep, my flow is insane\nI'm the greatest of all time, remember the name\n\n(Verse 2)\nI've been through hell and back, I've seen it all\nBut I never let it break me, I never fall\nI rise above the hate, I conquer the doubt\nI'm a living legend, there's no way out\n\nI spit truth, I speak my mind\nI'm not afraid to leave the haters behind\nI'm a warrior, a fighter, a champion\nI'll never stop, I'll never run\n\n(Chorus)\nI'm Eminem, the rap sensation\nI'll leave you in awe, in admiration\nMy words cut deep, my flow is insane\nI'm the greatest of all time, remember th

In [190]:


customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [191]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [192]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [193]:

customer_response = chat(customer_messages)
customer_response.content

"Oh man, I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

In [194]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


prompt =  'Translate the next set of lines to {language}: My name is Yaqoob Abdulhakeem, the first born of my father and mother. And i live in duisburg, Germany '
prompt_template =  ChatPromptTemplate.from_template(prompt)

prompt_template.input_variables

full_prompt = ChatPromptTemplate.format_messages(prompt_template, language = 'Arabic')
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat(full_prompt).content





'اسمي يعقوب عبد الحكيم، الأول من والدي وأمي. وأنا أعيش في دويسبورغ، ألمانيا.'

In [195]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

prompt2 = "from this ```{text}``` answer this questions in a json format, actor: What is the name of the main actor, year: what is the year of release of the movie "
promptemplate = ChatPromptTemplate.from_template(prompt2)
text = """Power tells the story of James St. Patrick, an intelligent, smooth yet ruthless drug dealer who goes by the alias of "Ghost." He wishes to leave the criminal world to pursue legitimate business interests as a nightclub owner. St. Patrick aims to balance those two lives, while also avoiding police capture, trying to navigate his crumbling marriage and manage shifting economic alliances.

The show features James' family, which consists of his wife Tasha, twins Tariq and Raina and baby Yasmine. Power also follows James' criminal partner and best friend Tommy Egan, love interest and criminal prosecutor Angela Valdes, friend-turned-rival Kanan Stark, protege and rival Andre Coleman, and Angela's colleague, Cooper Saxe. Defense attorney Joe Proctor, district attorney John Mak and politician Rashad Tate also appear in the show's later seasons.Power is an American crime drama thriller television series created and produced by Courtney A. Kemp in collaboration with Curtis "50 Cent" Jackson.[1] It aired on the Starz network from June 7, 2014, to February 9, 2020.

Upon release, Power gained positive reviews and is one of Starz's highest-rated shows and one of cable's most-watched shows.[2][3] Prior to the fifth-season premiere, Starz renewed the show for a sixth and final season, which premiered on August 25, 2019.
"""

model =  ChatPromptTemplate.format_messages(promptemplate, text=text )

main_actor = ResponseSchema(name='main_actor',
                            description='What is the name of the main actor')
year = ResponseSchema(name='year',
                      description='what is the year of release of the movie?')
response_schemas = [main_actor, year]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)




In [196]:
prompt2 = "from this ```{text}``` answer this questions in a json format, actor: What is the name of the main actor, year: what is the year of release of the movie {format_instructions}"

In [197]:
format_instructions = output_parser.get_format_instructions()
promptemplate_2 = ChatPromptTemplate.from_template(prompt2)
messages = promptemplate_2.format_messages(text=text, format_instructions = format_instructions)
output_parser.parse(chat(messages).content)

{'main_actor': 'Omari Hardwick', 'year': '2014'}

In [198]:
output_parser.parse(chat(messages).content).get('main_actor')

'Omari Hardwick'